In [ ]:
from keplergl import KeplerGl
import pandas as pd

In [ ]:
'''
Define paths and parameters
'''
DATA_PATH = './17_18.csv'

GEO_DATA_PATH = './country_lat_long.csv'

In [ ]:
# load dataset
DATASET = pd.read_csv(DATA_PATH, sep=';')

# replace Datum Uhrzeit with proper timestamp as index
DATASET['timestamp'] = DATASET.Datum.map(str) + ' ' + DATASET.Uhrzeit
DATASET = DATASET.drop(['Uhrzeit', 'Datum'], axis=1)
DATASET.set_index('timestamp', inplace=True)
# transform to time series
DATASET.index = pd.to_datetime(DATASET.index)
DATASET = DATASET.apply(pd.to_numeric, errors='coerce')
DATASET.fillna(0, inplace=True)

# load geo data
GEO_DATA = pd.read_csv(GEO_DATA_PATH, sep=';')
print(GEO_DATA)

In [ ]:
'''
Define data operation functions
'''

def shrinkIntervalMean(data, interval):
    '''
    returns the resampled dataset with the mean of the timestamp values in the given interval, 
    e.g. '10H' for 10 hour intervals
    '''
    return data.resample(interval).mean()

def toKeplerData(dataset, geodata):
    '''
    returns the given time series with each column as a distinct timestamp 
    in the series with the specific lat/lang attached
    '''
    series = pd.DataFrame()
    columns = list(dataset.columns)
    for index, row in dataset.iterrows():
        for col in columns:
            geo = geodata.loc[geodata['country'] == col]
            newTimestamp = pd.DataFrame({'value':row[col],'lat':geo.iloc[0]['lat'],
                                         'long':geo.iloc[0]['long']},index=[index])
            series = series.append(newTimestamp)

    return series

'''
Test functions
'''
DATASET = shrinkTimeGran(DATASET, '5H')
print(toKeplerData(DATASET, GEO_DATA).head(5))

In [ ]:
'''
Data preparation
'''

# reduce time series granularity to 5 hours
DATASET = shrinkTimeGran(DATASET, '5H')

In [4]:
'''
- Define Geo Data Location for data visualization
- Welches Format brauchen die Geo-Daten?
- Wo bekommen wir die Daten her?
- Wie macht man die Balken und Blasen mit kepler?

https://www.latlong.net
'''

map_1 = KeplerGl(height=500)

import_export_data = pd.read_csv("./test_data.csv", sep=";")
map_1.add_data(data=import_export_data, name='Strom Import/Export')

map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Strom Import/Export': {'index': [0, 1, 2, 3], 'columns': ['timestamp', 'lat', 'long', 'weight'…